In [ ]:
# Data Cleaning Notebook - Phelps et al. 2016 Glass Dataset
# Simple data cleaning procedures before analysis

# ============================================================================
# CELL 1: Import Libraries
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Libraries imported successfully")
print("Starting data cleaning process...")


In [ ]:
# ============================================================================
# CELL 2: Load Raw Data
# ============================================================================

# Load the Excel file
file_path = '/app/data/raw/phelps_et_al_2016.xlsx'

try:
    df_raw = pd.read_excel(file_path)
    print(f"✅ Data loaded successfully")
    print(f"Original dataset shape: {df_raw.shape}")
except FileNotFoundError:
    print("❌ File not found. Check the file path.")
    print("Available files in data/raw/:")
    print(list(Path('/app/data/raw/').glob('*')))

# Display first few rows
print("\nFirst 5 rows of raw data:")
df_raw.head()

In [ ]:
# ============================================================================
# CELL 3: Initial Data Inspection
# ============================================================================

print("="*60)
print("INITIAL DATA INSPECTION")
print("="*60)

# Basic info
print(f"Dataset shape: {df_raw.shape}")
print(f"Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Column information
print(f"\nColumn names and types:")
for i, (col, dtype) in enumerate(zip(df_raw.columns, df_raw.dtypes), 1):
    print(f"{i:2d}. {col:<25} ({dtype})")

# Check for duplicate column names
duplicate_cols = df_raw.columns[df_raw.columns.duplicated()].tolist()
if duplicate_cols:
    print(f"\n⚠️ Duplicate column names found: {duplicate_cols}")
else:
    print(f"\n✅ No duplicate column names")

In [ ]:
# ============================================================================
# CELL 4: Column Name Standardization
# ============================================================================

print("\n" + "="*60)
print("COLUMN NAME STANDARDIZATION")
print("="*60)

# Create a copy for cleaning
df_clean = df_raw.copy()

# Store original column names for reference
original_columns = df_clean.columns.tolist()

# Standardize column names: lowercase, remove spaces, replace special chars
df_clean.columns = (df_clean.columns
                   .str.lower()                    # Convert to lowercase
                   .str.replace(' ', '_')          # Replace spaces with underscores
                   .str.replace('-', '_')          # Replace hyphens with underscores
                   .str.replace('(', '')           # Remove parentheses
                   .str.replace(')', '')
                   .str.replace('%', 'pct')        # Replace % with pct
                   .str.replace('₂', '2')          # Replace subscript 2
                   .str.replace('₃', '3')          # Replace subscript 3
                   .str.strip('_'))                # Remove leading/trailing underscores

# Show column name changes
print("Column name changes:")
for old, new in zip(original_columns, df_clean.columns):
    if old != new:
        print(f"  '{old}' → '{new}'")
    
print(f"\nStandardized column names:")
for i, col in enumerate(df_clean.columns, 1):
    print(f"{i:2d}. {col}")


In [ ]:
# ============================================================================
# CELL 5: Missing Values Analysis
# ============================================================================

print("\n" + "="*60)
print("MISSING VALUES ANALYSIS")
print("="*60)

# Calculate missing values
missing_data = df_clean.isnull().sum()
missing_percent = (missing_data / len(df_clean)) * 100

# Create missing values summary
missing_summary = pd.DataFrame({
    'Column': missing_data.index,
    'Missing_Count': missing_data.values,
    'Missing_Percentage': missing_percent.values,
    'Data_Type': df_clean.dtypes.values
}).sort_values('Missing_Count', ascending=False)

# Show columns with missing values
columns_with_missing = missing_summary[missing_summary['Missing_Count'] > 0]

if len(columns_with_missing) > 0:
    print("Columns with missing values:")
    print(columns_with_missing.to_string(index=False))
    
    # Visualize missing data pattern
    plt.figure(figsize=(12, 6))
    
    # Missing values heatmap
    plt.subplot(1, 2, 1)
    missing_cols = columns_with_missing.head(20)['Column'].tolist()
    if missing_cols:
        sns.heatmap(df_clean[missing_cols].isnull(), 
                   cbar=True, yticklabels=False, cmap='viridis')
        plt.title('Missing Values Pattern')
        plt.xticks(rotation=45)
    
    # Missing values bar chart
    plt.subplot(1, 2, 2)
    top_missing = columns_with_missing.head(10)
    plt.barh(top_missing['Column'], top_missing['Missing_Percentage'])
    plt.xlabel('Missing Percentage (%)')
    plt.title('Top 10 Columns by Missing Data')
    plt.gca().invert_yaxis()
    
    plt.tight_layout()
    plt.show()
    
else:
    print("✅ No missing values found in the dataset")

In [ ]:
# ============================================================================
# CELL 6: Duplicate Rows Analysis
# ============================================================================

print("\n" + "="*60)
print("DUPLICATE ROWS ANALYSIS")
print("="*60)

# Check for complete duplicates
total_duplicates = df_clean.duplicated().sum()
print(f"Complete duplicate rows: {total_duplicates}")

if total_duplicates > 0:
    print("\nDuplicate rows found:")
    duplicate_rows = df_clean[df_clean.duplicated(keep=False)]
    print(duplicate_rows)
    
    # Option to remove duplicates
    df_clean = df_clean.drop_duplicates()
    print(f"After removing duplicates: {df_clean.shape}")
else:
    print("✅ No duplicate rows found")

# Check for duplicates in key identifier columns
id_columns = [col for col in df_clean.columns if 'id' in col.lower() or 'sample' in col.lower()]
if id_columns:
    print(f"\nChecking identifier columns: {id_columns}")
    for col in id_columns:
        duplicates = df_clean[col].duplicated().sum()
        if duplicates > 0:
            print(f"  {col}: {duplicates} duplicate values")
        else:
            print(f"  {col}: ✅ No duplicates")

In [ ]:
# ============================================================================
# CELL 7: Data Types Analysis and Cleaning
# ============================================================================

print("\n" + "="*60)
print("DATA TYPES ANALYSIS")
print("="*60)

# Show current data types
print("Current data types:")
print(df_clean.dtypes.value_counts())

# Identify numeric columns that might be stored as text
print(f"\nColumns analysis:")
for col in df_clean.columns:
    dtype = df_clean[col].dtype
    unique_count = df_clean[col].nunique()
    sample_values = df_clean[col].dropna().head(3).tolist()
    
    print(f"{col:<20} | {str(dtype):<10} | {unique_count:>3} unique | Sample: {sample_values}")

# Convert object columns to numeric where appropriate
print(f"\nAttempting to convert object columns to numeric...")

numeric_conversions = 0
for col in df_clean.columns:
    if df_clean[col].dtype == 'object':
        # Try to convert to numeric
        try:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
            numeric_conversions += 1
            print(f"  ✅ Converted {col} to numeric")
        except:
            print(f"  ⚠️ Could not convert {col} to numeric (keeping as text)")

print(f"Converted {numeric_conversions} columns to numeric")


In [ ]:
# ============================================================================
# CELL 8: Outlier Detection
# ============================================================================

print("\n" + "="*60)
print("OUTLIER DETECTION")
print("="*60)

# Get numeric columns
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
print(f"Numeric columns for outlier analysis: {len(numeric_cols)}")

if numeric_cols:
    # Calculate outliers using IQR method
    outlier_summary = []
    
    for col in numeric_cols:
        if df_clean[col].notna().sum() > 0:  # Only if column has data
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = df_clean[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)][col]
            outlier_count = len(outliers)
            
            outlier_summary.append({
                'Column': col,
                'Outlier_Count': outlier_count,
                'Outlier_Percentage': (outlier_count / len(df_clean)) * 100,
                'Min_Value': df_clean[col].min(),
                'Max_Value': df_clean[col].max(),
                'Lower_Bound': lower_bound,
                'Upper_Bound': upper_bound
            })
    
    outlier_df = pd.DataFrame(outlier_summary).sort_values('Outlier_Count', ascending=False)
    
    # Show columns with outliers
    outliers_found = outlier_df[outlier_df['Outlier_Count'] > 0]
    if len(outliers_found) > 0:
        print("Columns with outliers:")
        print(outliers_found.to_string(index=False))
    else:
        print("✅ No outliers detected using IQR method")

In [ ]:
# ============================================================================
# CELL 9: Data Consistency Checks
# ============================================================================

print("\n" + "="*60)
print("DATA CONSISTENCY CHECKS")
print("="*60)

# Check for negative values in oxide/element columns (should be positive)
oxide_element_cols = [col for col in numeric_cols if any(x in col.lower() for x in ['o', 'sio', 'cao', 'al', 'na', 'mg', 'fe', 'ti', 'k'])]

negative_checks = []
for col in oxide_element_cols:
    negative_count = (df_clean[col] < 0).sum()
    if negative_count > 0:
        negative_checks.append({'Column': col, 'Negative_Values': negative_count})

if negative_checks:
    print("⚠️ Columns with negative values (may be data entry errors):")
    for check in negative_checks:
        print(f"  {check['Column']}: {check['Negative_Values']} negative values")
else:
    print("✅ No negative values in chemical composition columns")

# Check for unrealistic percentages (>100% in oxide data)
percentage_checks = []
for col in oxide_element_cols:
    if df_clean[col].notna().sum() > 0:
        max_val = df_clean[col].max()
        if max_val > 100:
            percentage_checks.append({'Column': col, 'Max_Value': max_val})

if percentage_checks:
    print("⚠️ Columns with values >100% (check if these are percentages):")
    for check in percentage_checks:
        print(f"  {check['Column']}: max value = {check['Max_Value']:.2f}")
else:
    print("✅ No unrealistic percentage values found")
